In [1]:
%matplotlib inline
import pandas as pd

In [5]:
df = pd.read_csv('data/clean_compras.csv')

In [6]:
df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

## Empezamos con una expansion de datos que tenemos, fecha por ejemplo

requerimos convertir a formato de tiempo la columna par despues poder manipular este tipo de objeto datetime

In [7]:
# convertimos la col fecha a formato de datetime
df['FECHA'] = pd.to_datetime(df['FECHA'])

In [10]:
#df['DIA'] = df['FECHA'].apply(lambda x: x.day) # forma larga usando func lambda a cada registro 
df['DIA'] = df['FECHA'].dt.day # forma reducida usando el metodo dt
df['MES'] = df['FECHA'].dt.month
df['YEAR'] = df['FECHA'].dt.year
df.head(1)

,NUM BEN,BENEFICIARIO,RFC,FACTURA,CONTRATO,CONCEPTO,TIPO,# OP,FECHA,IMPORTE,SHEET,DIA,MES,YEAR
0,110152,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",LSS841015DV3,LLS AF8458-158008-3972,OT. 20204373,REEMPLAZO E INSTALACION DE AMORTIGUADORES (DEL...,TR,1019110.0,2020-12-11,14911.8,GASTOS VARIOS,11,12,2020


## Proyectemos unas variables contra otras
Prácticamente es guardar el valor de un groupby u otra función de agregación en una nueva columna 

In [30]:
beneficiario_sales = df['BENEFICIARIO'].value_counts().to_dict()
list(beneficiario_sales.items())[20:22]

[('LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.', 13),
 ('MILENIO DIARIO, S.A. DE C.V.', 12)]

### usar diccionario para asignar valores en lmbda
podemos usar una funcion lambda dentro de una columna un numero corresponiete a la llave de un diccionario. 
dicho de otra forma cada vez que x sea == dict.key x se le asignara el dict value correspondiente. 

In [33]:
df['BENEFICIARIO_SALES'] = df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x])
df.head(3)[["BENEFICIARIO", "BENEFICIARIO_SALES"]] # podemos ver como la nueva columna asigno el valor del dict de arriba

,BENEFICIARIO,BENEFICIARIO_SALES
0,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",13
1,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",13
2,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",13


In [36]:
# hacemos lo mismo de atras pero ahora con los valores de group by
sheet_mean_compras = df.groupby('SHEET')['IMPORTE'].mean().to_dict()
list(sheet_mean_compras.items())[:2]

[('COMUNICACION', 109705.34366197183),
 ('CONTRATISTAS Y FDO FED', 474566.76578729285)]

In [37]:
df['IMPORTE_GASTO_PROMEDIO'] = df['SHEET'].apply(lambda x: sheet_mean_compras[x]) 
df.head(3)[["BENEFICIARIO","SHEET", "IMPORTE_GASTO_PROMEDIO"]]

,BENEFICIARIO,SHEET,IMPORTE_GASTO_PROMEDIO
0,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",GASTOS VARIOS,881659.731797
1,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",GASTOS VARIOS,881659.731797
2,"LLANTAS Y SERVICIOS SERNA ANAHUAC,S.A. DE C.V.",GASTOS VARIOS,881659.731797


## Traernos informacion externa, enriquecer los datos usando una tool de paginas de linkedin

# phantom buster
en este caso se uso phantom buster para hacer scraping en linkedin. practicamente lo que hace es usar una lista de nombres para hacer scraping en linkedin. en este caso solo queremos confirmar si losbeneficiarios tienen  linkedin.   
para efectos de esta clase ya tendremos el dataset que tenga la informacion que requerimos, ya que el scraping sale del alcance del curso. 

https://phantombuster.com/phantombuster?category=linkedin


In [38]:
# creamos un CSV que contiene los nombres unicos de beneficiarios 
pd.DataFrame(df['BENEFICIARIO'].unique()).to_csv('data/unique_beneficiarios.csv', index=None)

In [39]:
#Cargar la data del scraping
linkedin_mty = pd.read_csv('data/linkedin_mty.csv')
linkedin_mty.head(2)

,url,description,title,query,timestamp,error
0,https://www.linkedin.com/in/rosa-lorena-valdez...,Ve el perfil de Rosa Lorena Valdez Miranda en ...,Rosa Lorena Valdez Miranda - Analista de Deuda...,Rosa Lorena Valdez Miranda,2021-03-08T13:06:25.587Z,NaN
1,https://www.linkedin.com/in/roque-ya%C3%B1ez-r...,"Ve el perfil de Roque Yañez Ramos en LinkedIn,...",Roque Yañez Ramos - Síndico Primero - Gobierno...,Roque Yañez Ramos,2021-03-08T13:06:52.599Z,NaN


In [45]:
has_site = {}
# fillna: llena los null con True. # iterrrow: saca cada fila como un dataframe con un solo registro
for i, ele in linkedin_mty.fillna(True).iterrows(): 
    has_site[ele['query']] = True if ele['error'] == True else False

list(has_site.items())[25:30]

[('DISTRIBUIDORA DE FRUTAS Y LEGUMBRES LA HORTALIZA,S.A.DE C.V.', False),
 ('NUÑEZ LOPEZ VIRGINIA', True),
 ('S.I.M.E.P.R.O.D.E.', False),
 ('TRACTORES Y MAQUINARIA REAL, S.A. DE C.V.', True),
 ('TELEFONOS DE MEXICO,S.A.B DE C.V.', True)]

In [46]:
# creamos la nueva columna, usando el diccionario rellenamos. 
#  cada registro de BENEFICIARIO igualamos la limpieza cambiando dobles espacios por 1
df['HAS_LINKEDIN_SITE'] = df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [47]:
df['HAS_LINKEDIN_SITE'].value_counts()

True     1392
False     148
Name: HAS_LINKEDIN_SITE, dtype: int64

In [49]:
# guardamos el DS enriquecido
df.to_csv('data/compras_df_enriched.csv', index=None)